# Minimum Code for Training the Model with Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

## Model Architecture

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D, BatchNormalization, Flatten, Dense, Dropout, MaxPool2D, LeakyReLU

def defineModel():
    """ Creates a sequetial model, defines it's architecture and returns it. """
    print("\n> Defining the model...")

    # - convolutional neural network -
    model = Sequential()

    model.add(Convolution2D(32, (3, 3), padding='same', use_bias=False, input_shape=(96, 96, 1)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Convolution2D(32, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Convolution2D(64, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Convolution2D(96, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Convolution2D(96, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Convolution2D(128, (3, 3), padding='same', use_bias=False))
    # model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Convolution2D(128, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Convolution2D(256, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Convolution2D(256, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Convolution2D(512, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Convolution2D(512, (3, 3), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(30))

    return model

## Training

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt

def trainModel(model, X_train, y_train, epochs=50, batch_size=256, validation_split=0.2, showHistory=True):
    """Compiles and fits the model inplace plus saves it if specified under the given name. """
    print("\n> Training the model...")
    print(f"> Epochs: {epochs}, Batch size: {batch_size}, Validation split: {validation_split}")

    model.compile(
        optimizer='adam',   # stochastic gradient descent
        loss=masked_mean_squared_error,
        metrics=[masked_mean_absolute_error, masked_accuracy],
        run_eagerly=True    # custom metrics
    )

    history = model.fit(
        X_train,
        y_train,
        epochs=epochs,                      # number of epochs: 50
        batch_size=batch_size,              # batch size of 256
        validation_split=validation_split   # 20% of data for validation
    )

    if showHistory:
        showTrainingHistory(history)


def filter_mask(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_true, tf.constant(-1.)))
    y_true = tf.boolean_mask(y_true, mask)
    y_pred = tf.boolean_mask(y_pred, mask)
    return y_true, y_pred


def masked_mean_squared_error(y_true, y_pred):
    y_true, y_pred = filter_mask(y_true, y_pred)
    loss = tf.square(y_true - y_pred)  # Mean Squared Error
    return tf.reduce_mean(loss)


def masked_mean_absolute_error(y_true, y_pred):
    y_true, y_pred = filter_mask(y_true, y_pred)
    loss = tf.abs(y_true - y_pred)  # Mean Absolute Error
    return tf.reduce_mean(loss)

def masked_accuracy(y_true, y_pred):
    y_true, y_pred = filter_mask(y_true, y_pred)
    diff = tf.abs(y_true - y_pred)
    passed = tf.math.count_nonzero(diff < 2)  # in margin of one pixel the prediction is counted as correct
    masked_accuracy.inMargin += passed.numpy()
    masked_accuracy.total += tf.size(diff).numpy()
    return masked_accuracy.inMargin / masked_accuracy.total


masked_accuracy.inMargin = 0
masked_accuracy.total = 0


def saveModel(model, path="../../models/modelV1"):
    """Saves the model to the given path."""
    print("\n> Saving the model...")
    model.save(path)
    print("Model saved to: " + path)

global historyPlot
def showTrainingHistory(history):
    """Shows the training history of the model."""
    keys = list(history.history.keys())  # for example ['loss', 'accuracy', 'val_loss', 'val_accuracy']
    mid = int(len(keys) / 2)
    fig, axes = plt.subplots(1, mid)

    for i, ax in enumerate(axes):
        ax.plot(history.history[keys[i]])
        ax.plot(history.history[keys[i + mid]])
        ax.set_ylabel(keys[i])
        ax.set_xlabel('epochs')
        ax.legend(['train', 'validation'], loc='best')

    global historyPlot
    historyPlot = fig

## Main

In [ ]:
import numpy as np

# Enter the path to the files
X_train = np.load("/content/drive/MyDrive/Module/05 Deep Learning/DeepLearning-FacialLandmarkDetection/data/processedData/X_train.npy").astype(np.float32)
y_train = np.load('/content/drive/MyDrive/Module/05 Deep Learning/DeepLearning-FacialLandmarkDetection/data/processedData/y_train.npy')

model = defineModel()
trainModel(
    model, X_train, y_train,
    epochs=100,
    batch_size=256,
    validation_split=0.2,
    showHistory=True
)

# Enter the path to save the model
saveModel(model, "/content/drive/MyDrive/Module/05 Deep Learning/DeepLearning-FacialLandmarkDetection/models/modelV2")

In [ ]:
historyPlot.set_dpi(300)
historyPlot.set_size_inches(12, 4)
historyPlot.set_facecolor("white")
historyPlot.set_tight_layout(True)
historyPlot